In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
file_list = [f for f in os.listdir("C:/Users/weizi/OneDrive - Texas A&M University/Publishing_Paper/PDE_Network_Modeling/2_RawData/District6_Raw") if f.endswith('.txt')]

In [3]:
print(file_list)

['d06_text_station_raw_2023_08_15.txt', 'd06_text_station_raw_2023_08_16.txt', 'd06_text_station_raw_2023_08_17.txt', 'd06_text_station_raw_2023_08_18.txt', 'd06_text_station_raw_2023_08_19.txt', 'd06_text_station_raw_2023_08_20.txt', 'd06_text_station_raw_2023_08_21.txt']


In [4]:
delimiter = ','  # Change this to the actual delimiter if it's different
data_all = pd.DataFrame()
for file_path in file_list:
    # Read the text file into a Pandas DataFrame
    df = pd.read_csv('C:/Users/weizi/OneDrive - Texas A&M University/Publishing_Paper/PDE_Network_Modeling/2_RawData/District6_Raw/' + file_path, delimiter=delimiter, header=None)
    # df = df.iloc[:, :12]
    # names = ['TimeStamp', 'Station', 'District', 'Freeway_#', 'Dir_Travel', 'Type', 'Length', 'Samples',
    #          'Observed_%', 'Total_Flow', 'Occupancy', 'Speed']
    names = ['TimeStamp', 'Station', 'flow_0', 'occu_0', 'speed_0', 
             'flow_1', 'occu_1', 'speed_1', 'flow_2', 'occu_2', 'speed_2',
             'flow_3', 'occu_3', 'speed_3','flow_4', 'occu_4', 'speed_4',
             'flow_5', 'occu_5', 'speed_5', 'flow_6', 'occu_6', 'speed_6', 
             'flow_7', 'occu_7', 'speed_7']
    df.columns = names
    # df = df[df['Type']=="ML"]

    data_all = pd.concat([data_all, df], ignore_index=True)

In [5]:
# data_all_observed = data_all[data_all['Observed_%'] > 0]
# summary_observed = data_all_observed.groupby('Station').size().reset_index(name='Count')

In [6]:
# metadata = pd.read_csv('Data_Raw/Archive/d06_text_meta_2023_08_08.txt', delimiter='\t')
# metadata = metadata[metadata['Type']=="ML"]

In [7]:
# metadata_obs = pd.merge(metadata, summary_observed, left_on='ID', right_on='Station' , how='left')

In [8]:
# data_all_observed

## Station Selection

In [9]:
# metadata_obs.to_csv('Data_Raw/Station_MetaData_d06.csv', index=False)

In [10]:
station_selected = pd.read_csv('Data_Raw/I5N_Exported.csv')
station_selected = station_selected[station_selected['Count'] == 2016]

In [12]:
station_selected['PM_Dir'] = station_selected['Abs_PM'] - station_selected['Abs_PM'].min()
#station_selected['PM_Dir_round0.5'] = station_selected['PM_Dir'].apply(lambda x: round(x * 2) / 2)
#station_selected['Station_PM'] = station_selected['PM_Dir'].round(0)

station_selected['Station_PM'] = np.round(station_selected['PM_Dir'] / 2) * 2
station_selected['PM_Deviation'] = abs(station_selected['PM_Dir'] - station_selected['Station_PM'])

In [13]:
# select the stations where the deviation from integer mile is the smallest.
min_index = station_selected.groupby('Station_PM')['PM_Deviation'].idxmin()
station_selected = station_selected.loc[min_index]

In [14]:
station_selected['location_step'] = station_selected['Station_PM']/2
station_selected['Station_Number'] = station_selected['Station_PM'].rank(ascending=True)

In [15]:
# select the first 60 miles of I-5 North
station_selected = station_selected[(station_selected['Station_PM'] < 60) & (station_selected['Station_PM'] > 20)]

In [16]:
station_selected.to_csv('Data_Raw/Station_Selected_I5N.csv', index=False)

In [17]:
order_match = station_selected[['ID', 'location_step']]
order_match = order_match.rename(columns={'location_step': 'Station_PM'})

In [18]:
ID_selected = station_selected['ID']
data_all_select = data_all[data_all['Station'].isin(ID_selected)].reset_index(drop=True)
data_all_select.loc[:, 'TimeStamp'] = pd.to_datetime(data_all_select['TimeStamp'])

In [19]:
data_all_select.loc[:,'flow'] = data_all_select.filter(like='flow_').sum(axis=1)

In [20]:
data_all_select.loc[:,'occu'] = data_all_select.filter(like='occu_').mean(axis=1)

In [21]:
weighted_speed = data_all_select[[f'speed_{i}' for i in range(8)]].multiply(data_all_select[[f'flow_{i}' for i in range(8)]].values, axis=0)
data_all_select.loc[:,'speed'] = weighted_speed.sum(axis=1, skipna=True) / data_all_select[[f'flow_{i}' for i in range(8)]].sum(axis=1, skipna=True)

In [22]:
data_all_select = data_all_select[['TimeStamp', 'Station', 'flow', 'occu', 'speed']]
data_all_select['TimeStamp'] = pd.to_datetime(data_all_select['TimeStamp'])
data_all_select.set_index('TimeStamp', inplace=True)

In [23]:
data_all_select['flow_speed'] = data_all_select['flow'] * data_all_select['speed']

In [24]:
# Regroup flow, occu, flow measurements
data_all_select_resampled = data_all_select.groupby('Station').resample('3min').agg({
    'flow': 'sum',        # Sum for flow
    'occu': 'mean',   # Average for occupancy
    'flow_speed': 'sum'
})

In [25]:
data_all_select_resampled['speed'] = data_all_select_resampled['flow_speed']/data_all_select_resampled['flow']
data_all_select_resampled = data_all_select_resampled.drop(columns=['flow_speed'])
data_all_select_resampled.reset_index(inplace=True)

In [26]:
data_all_select_resampled['TimeDiff'] = ((data_all_select_resampled['TimeStamp'] - data_all_select_resampled['TimeStamp'].min()).dt.total_seconds())/60
data_all_select_resampled['TimeStep'] = 1 + (data_all_select_resampled['TimeDiff'] // 3)

In [27]:
#station_selected[station_selected['PM_Dir_round'].duplicated(keep=False)]

In [28]:
data_all_select = pd.merge(data_all_select_resampled, order_match, left_on='Station', right_on='ID' , how='left')

In [30]:
Flow = data_all_select[['TimeStep', 'Station_PM', 'flow']]
Flow['TimeStep'] = Flow['TimeStep'].astype(int)
Flow['Station_PM'] = Flow['Station_PM'].astype(int)

Occupancy = data_all_select[['TimeStep', 'Station_PM', 'occu']]
Occupancy['TimeStep'] = Occupancy['TimeStep'].astype(int)
Occupancy['Station_PM'] = Occupancy['Station_PM'].astype(int)

Speed = data_all_select[['TimeStep', 'Station_PM', 'speed']]
Speed['TimeStep'] = Speed['TimeStep'].astype(int)
Speed['Station_PM'] = Speed['Station_PM'].astype(int)

C:\Users\weizi\AppData\Local\Temp\ipykernel_26892\2773514643.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Flow['TimeStep'] = Flow['TimeStep'].astype(int)
C:\Users\weizi\AppData\Local\Temp\ipykernel_26892\2773514643.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Flow['Station_PM'] = Flow['Station_PM'].astype(int)
C:\Users\weizi\AppData\Local\Temp\ipykernel_26892\2773514643.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [31]:
# renumber the station PM field from 0.
Flow['Station_PM'] = Flow['Station_PM'] - Flow['Station_PM'].min()
Occupancy['Station_PM'] = Occupancy['Station_PM'] - Occupancy['Station_PM'].min()
Speed['Station_PM'] = Speed['Station_PM'] - Speed['Station_PM'].min()

C:\Users\weizi\AppData\Local\Temp\ipykernel_26892\170357285.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Flow['Station_PM'] = Flow['Station_PM'] - Flow['Station_PM'].min()
C:\Users\weizi\AppData\Local\Temp\ipykernel_26892\170357285.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Occupancy['Station_PM'] = Occupancy['Station_PM'] - Occupancy['Station_PM'].min()
C:\Users\weizi\AppData\Local\Temp\ipykernel_26892\170357285.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

## Final Data Cleaning

In [32]:
"""
Create a table with timestep and PM (posted Miles) as one-one match
"""
timestep_value = [i for i in range(1, Occupancy['TimeStep'].max() + 1) for _ in range(Occupancy['Station_PM'].max() + 1)]
PM_values = list(range(Occupancy['Station_PM'].max() + 1)) * Occupancy['TimeStep'].max()
Time_PM = pd.DataFrame({'TimeStep': timestep_value, 'Station_PM': PM_values})

In [33]:
Occupancy_all = pd.merge(Time_PM, Occupancy, on=['TimeStep', 'Station_PM'] , how='left')
Flow_all = pd.merge(Time_PM, Flow, on=['TimeStep', 'Station_PM'] , how='left')
Speed_all = pd.merge(Time_PM, Speed, on=['TimeStep', 'Station_PM'] , how='left')

In [34]:
Occupancy_all_wide = Occupancy_all.pivot(index='Station_PM', columns='TimeStep', values='occu')
Flow_all_wide = Flow_all.pivot(index='Station_PM', columns='TimeStep', values='flow')
Speed_all_wide = Speed_all.pivot(index='Station_PM', columns='TimeStep', values='speed')

In [35]:
Flow_wide = Flow.pivot(index='Station_PM', columns='TimeStep', values='flow')
Occupancy_wide = Occupancy.pivot(index='Station_PM', columns='TimeStep', values='occu')
Speed_wide = Speed.pivot(index='Station_PM', columns='TimeStep', values='speed')

In [38]:
Flow_wide

TimeStep,1,2,3,4,5,6,7,8,9,10,...,3351,3352,3353,3354,3355,3356,3357,3358,3359,3360
Station_PM,,,,,,,,,,,,,,,,,,,,,
0,19.0,34.0,27.0,23.0,17.0,24.0,23.0,18.0,21.0,20.0,...,24.0,21.0,25.0,21.0,15.0,28.0,20.0,14.0,23.0,27.0
1,21.0,26.0,35.0,19.0,16.0,16.0,22.0,25.0,15.0,27.0,...,20.0,20.0,20.0,31.0,18.0,27.0,20.0,15.0,18.0,23.0
4,17.0,18.0,21.0,14.0,36.0,33.0,18.0,20.0,19.0,15.0,...,17.0,25.0,20.0,23.0,22.0,28.0,20.0,25.0,22.0,18.0
7,18.0,21.0,26.0,22.0,19.0,27.0,32.0,16.0,24.0,15.0,...,18.0,19.0,19.0,30.0,20.0,16.0,25.0,28.0,16.0,31.0
9,38.0,27.0,25.0,16.0,36.0,20.0,25.0,29.0,27.0,14.0,...,27.0,29.0,22.0,24.0,20.0,27.0,21.0,25.0,27.0,23.0
11,31.0,41.0,25.0,24.0,20.0,36.0,21.0,22.0,39.0,24.0,...,20.0,30.0,27.0,21.0,21.0,22.0,25.0,19.0,28.0,23.0
13,44.0,29.0,27.0,33.0,29.0,22.0,22.0,32.0,21.0,31.0,...,22.0,24.0,25.0,28.0,22.0,19.0,21.0,23.0,16.0,21.0
14,35.0,41.0,28.0,34.0,32.0,24.0,18.0,26.0,25.0,29.0,...,13.0,28.0,24.0,32.0,20.0,21.0,15.0,28.0,17.0,21.0
15,32.0,43.0,33.0,34.0,29.0,28.0,26.0,23.0,30.0,23.0,...,26.0,23.0,27.0,19.0,30.0,21.0,24.0,18.0,21.0,21.0


In [42]:
Flow_all_wide.to_csv('Data_Clean/Flow_all.csv')
Occupancy_all_wide.to_csv('Data_Clean/Occupancy_all.csv')
Speed_all_wide.to_csv('Data_Clean/Speed_all.csv')

In [43]:
Flow_wide.to_csv('Data_Clean/Flow.csv')
Occupancy_wide.to_csv('Data_Clean/Occupancy.csv')
Speed_wide.to_csv('Data_Clean/Speed.csv')

In [44]:
Flow.to_csv('Data_Clean/Flow_Long.csv', index=False)
Occupancy.to_csv('Data_Clean/Occupancy_Long.csv', index=False)
Speed.to_csv('Data_Clean/Speed_Long.csv', index=False)

In [45]:
Flow_all.to_csv('Data_Clean/Flow_all_Long.csv', index=False)
Occupancy_all.to_csv('Data_Clean/Occupancy_all_Long.csv', index=False)
Speed_all.to_csv('Data_Clean/Speed_all_Long.csv', index=False)